In [32]:
# Step 1: Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K

In [33]:
# Step 2: Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0  # Normalize to [0, 1]
x_test = x_test.astype("float32") / 255.0

# Reshape images to include the channel dimension
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [34]:
def make_pairs(images, labels):
    pairs = []
    labels_pairs = []
    num_classes = max(labels) + 1
    digit_indices = [np.where(labels == i)[0] for i in range(num_classes)]

    for idx1 in range(len(images)):
        x1, label1 = images[idx1], labels[idx1]

        # Create a positive pair
        idx2 = np.random.choice(digit_indices[label1])
        x2 = images[idx2]
        pairs.append([x1, x2])
        labels_pairs.append(1)  # Label for positive pair

        # Create a negative pair
        label2 = np.random.randint(0, num_classes)
        while label2 == label1:  # Ensure the label is different for negative pairs
            label2 = np.random.randint(0, num_classes)
        idx2 = np.random.choice(digit_indices[label2])
        x2 = images[idx2]
        pairs.append([x1, x2])
        labels_pairs.append(0)  # Label for negative pair

    return np.array(pairs), np.array(labels_pairs).astype("float32")

# Create pairs for training and testing
pairs_train, labels_train = make_pairs(x_train, y_train)
pairs_test, labels_test = make_pairs(x_test, y_test)

In [35]:
def create_base_network(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    return model

# Register contrastive loss
@tf.keras.utils.register_keras_serializable()
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    squared_pred = K.square(y_pred)
    squared_margin = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * squared_pred + (1 - y_true) * squared_margin)

# Register the euclidean_distance function
@tf.keras.utils.register_keras_serializable()
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


In [36]:
def siamese_network(input_shape):
    base_network = create_base_network(input_shape)

    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    output_a = base_network(input_a)
    output_b = base_network(input_b)

    # Calculate the Euclidean distance between embeddings
    def euclidean_distance(vects):
        x, y = vects
        return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

    distance = layers.Lambda(euclidean_distance, output_shape=(1,))([output_a, output_b])

    model = models.Model(inputs=[input_a, input_b], outputs=distance)
    return model

In [37]:
def siamese_network(input_shape):
    base_network = create_base_network(input_shape)

    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    output_a = base_network(input_a)
    output_b = base_network(input_b)

    distance = layers.Lambda(euclidean_distance, output_shape=(1,))([output_a, output_b])

    model = models.Model(inputs=[input_a, input_b], outputs=distance)
    return model

input_shape = (28, 28, 1)  # Shape of MNIST images
model = siamese_network(input_shape)
model.compile(loss=contrastive_loss, optimizer='adam')

In [38]:
def train_model(model, pairs, labels, epochs=20, batch_size=64):
    images_a = np.array([pair[0] for pair in pairs]).reshape(-1, 28, 28, 1)
    images_b = np.array([pair[1] for pair in pairs]).reshape(-1, 28, 28, 1)

    model.fit([images_a, images_b], labels,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2)  # Use a validation split

# Train the model
train_model(model, pairs_train, labels_train)

Epoch 1/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 150s 99ms/step - loss: 0.0724 - val_loss: 0.0260
Epoch 2/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 199s 97ms/step - loss: 0.0235 - val_loss: 0.0208
Epoch 3/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 200s 96ms/step - loss: 0.0178 - val_loss: 0.0175
Epoch 4/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 200s 95ms/step - loss: 0.0142 - val_loss: 0.0151
Epoch 5/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 203s 96ms/step - loss: 0.0123 - val_loss: 0.0146
Epoch 6/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 203s 97ms/step - loss: 0.0110 - val_loss: 0.0130
Epoch 7/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 200s 96ms/step - loss: 0.0097 - val_loss: 0.0123
Epoch 8/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 203s 97ms/step - loss: 0.0089 - val_loss: 0.0121
Epoch 9/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 203s 97ms/step - loss: 0.0081 - val_loss: 0.0121
Epoch 10/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 202s 97ms/step - loss: 0.0076 - val_loss: 0.0114
Epoch 11/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 199s 95ms/step - loss: 0.0072 - val_loss: 0.01

In [39]:
# Step 7: Save the model
model.save('siamese_network_model.h5')  # Save the model to a file
print("Model saved as 'siamese_network_model.h5'")

# Load the model
loaded_model = load_model('siamese_network_model.h5',
                          custom_objects={'contrastive_loss': contrastive_loss,
                                          'euclidean_distance': euclidean_distance})

# Evaluate the model
def evaluate_model(model, pairs, labels):
    images_a = np.array([pair[0] for pair in pairs]).reshape(-1, 28, 28, 1)
    images_b = np.array([pair[1] for pair in pairs]).reshape(-1, 28, 28, 1)

    predictions = model.predict([images_a, images_b])
    threshold = 0.5  # You might need to adjust this threshold based on your data
    predicted_labels = (predictions < threshold).astype(int)

    accuracy = np.mean(predicted_labels.flatten() == labels)
    print(f'Accuracy: {accuracy * 100:.2f}%')

# Evaluate the model using the test pairs
evaluate_model(loaded_model, pairs_test, labels_test)

Model saved as 'siamese_network_model.h5'
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step
Accuracy: 98.48%
